In [1]:
# Import der benötigten Libraries
import pandas as pd
import nltk
import numpy as np
import re
import os

pd.set_option('display.max_colwidth', 50)

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/JobCloud_Daten')
print(os.getcwd())

# Files im Directory
print(os.listdir())

C:\Users\Sven Konermann\Documents\Master\2_Semester\aF_E_Faelle\Unterlagen\JobCloud_Daten
['Anzahl_Inserate_Anforderungsdaten.pdf', 'Anzahl_Inserate_Gesamtdaten.pdf', 'ausgewertete_Anforderungen.csv', 'Daten_JobCloud001.txt', 'Daten_JobCloud002.txt', 'Daten_JobCloud003.txt', 'Fixwoerter.xlsx', 'Inserate_Anforderungsanalyse_AF_E.csv', 'Inserate_clean_Anforderungsanalyse_AF_E.csv', 'Inserate_clean_Gesamtdatenanalyse.csv', 'Inserate_clean_Titelanalyse_AF_E.csv', 'Inserate_Gesamtdatenanalyse_AF_E.csv', 'ngram_count_all_Data.csv', 'ngram_count_all_Title.csv', 'Phrasenwoerter.xlsx']


In [26]:
data = pd.read_csv("Inserate_clean_Gesamtdatenanalyse.csv", sep=";")
data = data[~data['Text_Anforderungen'].isnull()] # Wenn NULL-Werte, lösche diese
print(data.shape)

(132764, 16)


In [27]:
set_phrases = pd.read_excel('Phrasenwoerter.xlsx')
set_phrases.head()

,word,kategorie,kategorie_name,hauptkategorie
0,visual studio,2,Informatik Software-Kenntnisse,IT-Kenntnisse
1,big data,3,Datenmanagement,IT-Kenntnisse
2,business intelligence,3,Datenmanagement,IT-Kenntnisse
3,data science,3,Datenmanagement,IT-Kenntnisse
4,datenbank,3,Datenmanagement,IT-Kenntnisse


In [28]:
options =	{
    "Administration/HR/Consulting/CEO": ['Beratung/Unternehmensentwicklung','CEO/GeschÃ¤ftsfÃ¼hrung','Direktionsassistenz','Juristen/AnwÃ¤lte/Gericht','Personalvermittlung/-beratung','Personalwesen/-entwicklung','QualitÃ¤tsmanagement','Regulatory Affairs','Sachbearbeitung/SprachÃ¼bersetzung','Sekretariat/Empfang', 'None'],
    "Banking/Versicherungswesen": ["Mustang"'Aktuariat/Mathematik','Anlageberatung/Private Banking','Asset/Portfolio Management','Back-Office/Zahlungsverkehr','Finanz-/Business Analyse','Fonds/Wertschriften/Handel','Kredit/Hypotheken','Kundenberatung/Relationship Mgmt','Projekt Management','Risk Mgmt/Compliance','Treasury/Controlling/Audit','Underwriting/Product Mgmt','Versicherung: Agenturen','Versicherung: Akquisition/Beratung','Versicherung: Schaden/Leistungen', 'None'],
    "Bau/Architektur/Engineering": ['Architektur/Bauplanung/Zeichner','Bauleitung/Bauingenieure/AVOR','Baupersonal/GerÃ¼stbau/Maschinen','Bauspengler/Dachdecker','Boden-/Plattenleger','Elektro Planung/Montage','Gartenbau/Landschaftsarchitektur','Heizung/LÃ¼ftung/Klima Planung/Montage','Holzbau/Zimmermann','Innenarchitektur/Dekoration','Maler/Gipser','Maurer','Metall/Fassadenbau','SanitÃ¤r Planung/Montage', 'None'],
    "Bewachung/Polizei/Zoll/Rettung": ['Bewachung/Kontrolle/Sicherheit','Polizei/MilitÃ¤r','Rettung SanitÃ¤t/Feuerwehr','Zoll/Grenzkontrolle', 'None'],
    "Chemie/Pharma/Biotechnologie": ['Apotheken/Drogerien','Biologie/Biotechnologie','Chemie F+E/Analytik/Produktion','Pharma F+E/Analytik/Produktion','QualitÃ¤tswesen/-management', 'None'],
    "Einkauf/Logistik/Trading": ['Einkauf/Procurement','Import/Export/Trading','Logistik/Supply Chain', 'None'],
    "Elektronik/Technik/Uhren": ['Elektronik/Elektrotechnik','Energie-/GebÃ¤ude-/Haustechnik','Kundendienst/Beratung','Mechanik/Feinmechanik/Uhren','Medizinaltechnik','QualitÃ¤tswesen/-management','Servicetechnik','Werkzeugbau', 'None'],
    "Fahrzeuge/Handwerk/Lager/Transport": ['Carosseriespengler/Lackierer','Facility Management/Reinigung','Fahrzeugmechanik/-diagnostik','Holzhandwerk','Lager','Land-/Forstwirtschaft','Metallhandwerk','Personentransport Flug/See/Zug/Strasse','Textil/Schneider', 'None'],
    "Finanzen/Treuhand/Immobilien": ['Audit/Revision/WirtschaftsprÃ¼fung','Buchhaltung','Controlling','Finanz-/Rechnungswesen','Immobilien Handel/Bewirtschaftung','Inkasso','Lohnbuchhaltung','Steuern','Treuhand', 'None'],
    "Gastronomie/Lebensmittel/Tourismus": ['BetriebsfÃ¼hrung/-administration','KÃ¼che','Lebensmitteltechnologie/-ingenieure','Metzger/BÃ¤cker/Konditor/KÃ¤ser','RÃ©ception/Portier/Concierge','Reiseberatung/Reiseleitung','Service/Bar','Zimmer/Lingerie/Hauswirtschaft', 'None'],
    "Grafik/Typografie/Druck": ['Drucker','Grafik/Design','Typografie/Polygrafie', 'None'],
    "Informatik/Telekommunikation": ['Beratung/Wirtschaftsinformatik','Datenbank Spezialisten/Entwicklung','ERP/SAP/CRM','Netzwerk Spezialisten/Engineers','Projekt Management/Analyse','Software Architektur/Engineering','Software Programmierung','System Administration','System Engineering','Testing/Audit/Security','User Help Desk/Support/Training','Web Programmierung/Mobile','Web Publishing/Design', 'None'],
    "Marketing/Kommunikation/Redaktion": ['Marketing/Direktmarketing','Medien/Redaktion/Verlagswesen','Online Marketing/Social Media','PR/Corporate Communications','Produkt-/Brand Management','Werbeberatung/-kommunikation', 'None'],
    "Maschinen-/Anlagenbau/Produktion": ['Anlagen-/Apparatebau','Automation/Verfahrenstechnik','CNC-Mechanik/Polymechanik','Konstruktion','Maschinenbau','Produktion/Fertigung/Montage','QualitÃ¤tswesen/-management','Servicetechnik/Wartung', 'None'],
    "Medizin/Pflege/Therapie": ['Ã„rzte - Assistenz-/Fach-/OberÃ¤rzte','Akutpflege','Hebammen','Labor/Analyse','Langzeitpflege','Med. Praxisassistenz/Sekretariat','Psychologie/Psychiatrie','Radiologie','Therapie/OrthopÃ¤die','Tiermedizin', 'None'],
    "Sport/Wellness/Kultur": ['Coiffeur/Kosmetik','Kultur/Unterhaltung','Massage/Sport/Wellness','Mode/Film/Theater/Radio','MuseumKunst/Kunsthandwerk', 'None'],
    "Verkauf/Kundendienst/Innendienst": ['Category Management','Kundenberatung/Key Account Mgmt','Kundendienst/Call Center','Telefonverkauf/Telesale','Verkauf Aussendienst/Spezialisten','Verkauf Beratung/Management','Verkauf Detailhandel','Verkauf Innendienst/Back-Office', 'None'],
    "Verwaltung/Bildung/Soziales": ['Lehrer/Dozenten','NGO/NPO/Hilfswerke','Oeffentliche Verwaltung','Sozial-/HeilpÃ¤dagogen','Sozial-/Jugendarbeit','Wissenschaft/Forschung', 'None']
}

user_input = ''


input_message = "Pick an option:\n"

for index, item in enumerate(options):
    input_message += f'{index+1}) {item}\n'

input_message += 'Your choice: '

while user_input not in options:
    user_input = input(input_message)
print('You picked: ' + user_input)



Pick an option:
1) Administration/HR/Consulting/CEO
2) Banking/Versicherungswesen
3) Bau/Architektur/Engineering
4) Bewachung/Polizei/Zoll/Rettung
5) Chemie/Pharma/Biotechnologie
6) Einkauf/Logistik/Trading
7) Elektronik/Technik/Uhren
8) Fahrzeuge/Handwerk/Lager/Transport
9) Finanzen/Treuhand/Immobilien
10) Gastronomie/Lebensmittel/Tourismus
11) Grafik/Typografie/Druck
12) Informatik/Telekommunikation
13) Marketing/Kommunikation/Redaktion
14) Maschinen-/Anlagenbau/Produktion
15) Medizin/Pflege/Therapie
16) Sport/Wellness/Kultur
17) Verkauf/Kundendienst/Innendienst
18) Verwaltung/Bildung/Soziales
Your choice: Administration/HR/Consulting/CEO
You picked: Administration/HR/Consulting/CEO


In [29]:
#print(options[user_input])
#print(options["brand"])
#print(input_message_2) 
user_input_2 = ''
input_message_2 = "Pick an option:\n"

for index, item in enumerate(options[user_input]):
    input_message_2 += f'{index+1}) {item}\n'
    
input_message_2 += 'Your choice: '

while user_input_2 not in options[user_input]:
    user_input_2 = input(input_message_2)

Pick an option:
1) Beratung/Unternehmensentwicklung
2) CEO/GeschÃ¤ftsfÃ¼hrung
3) Direktionsassistenz
4) Juristen/AnwÃ¤lte/Gericht
5) Personalvermittlung/-beratung
6) Personalwesen/-entwicklung
7) QualitÃ¤tsmanagement
8) Regulatory Affairs
9) Sachbearbeitung/SprachÃ¼bersetzung
10) Sekretariat/Empfang
11) None
Your choice: None


In [30]:
print(user_input)
print(user_input_2)

Administration/HR/Consulting/CEO
None


In [31]:
data.head(10)

,Unnamed: 0.1,Unnamed: 0,ad_id,active_start_on,active_end_on,title,text,branchen_name,customer_branche_name,lvl1_region_name,region_name,lvl1_jobcategory_name,jobcategory_name,duration,year,Text_Anforderungen
0,0,5,jobsse-10107990,2021-01-07,2021-01-15,Lead Solution Architect IoT,Lead Solution Architect IoT Als Lead Solution ...,Informatik/Telekommunikation,Beratung diverse,Region Bern,Region Bern,Informatik/Telekommunikation,Software Architektur/Engineering,8,2021,lead solution architect iot lead solution arch...
1,1,7,jobsse-8132125,2018-04-17,2018-04-18,Konstrukteur/In (auch Teilzeit möglich),<p><strong>Ihre Hauptaufgaben</strong></p><ul>...,Gewerbe/Handwerk allgemein,Personalberatung,"Region Mittelland (AG, SO)","Region Mittelland (AG, SO)",Maschinen-/Anlagenbau/Produktion,Konstruktion,1,2018,hauptaufgabenteile verschieden softwareapplika...
2,2,11,jobsse-6058207,2014-10-06,2014-11-07,Kaufmännische(n) Angestellte(n) 80%-100%,<p><strong>Wir z&auml;hlen auf Sie:</strong></...,Banken/ Finanzinstitute,Banken/ Finanzinstitute,Region Zentralschweiz,Region Zentralschweiz,Administration/HR/Consulting/CEO,Sekretariat/Empfang,32,2014,zählen siesie führen teilbereich administratio...
3,3,13,jobsse-5875311,2014-06-17,2014-06-20,Hauswirtschaftsmitarbeiter (m/w),Das Widder Hotel Moderne Architektur in 9 hist...,Gastgewerbe/Hotellerie,Gastgewerbe/Hotellerie,NaN,NaN,Gastronomie/Lebensmittel/Tourismus,Zimmer/Lingerie/Hauswirtschaft,3,2014,widder hotel modern architektur 9 historisch a...
4,4,14,jobsse-6012460,2014-09-09,2014-09-16,Kindergärtner/in 85 %,"Kindergärtner/in, Ref.-Nr. 41427D Bezei...",öffentl. Verwaltung/ Verbände,öffentl. Verwaltung/ Verbände,NaN,NaN,Verwaltung/Bildung/Soziales,Lehrer/Dozenten,7,2014,kindergärtnerin refnr 41427d bezeichnung besch...
5,5,24,jobsse-6911861,2016-04-12,2016-04-15,Regionaldirektor Medical Ostschweiz (m/w),<p>Um in dieser verantwortungsvollen Aufgabe e...,Medizinaltechnik,Personalberatung,Region Zürich / Schaffhausen,Region Zürich / Schaffhausen,Verkauf/Kundendienst/Innendienst,Verkauf Beratung/Management,3,2016,verantwortungsvoll aufgabe erfolgreich bringen...
6,6,25,jobsse-7494499,2017-04-19,2017-05-21,Mitarbeiter/in Logistik 50%,"<p>Six Sigma Tools AG ist ein junges, schnell ...",Industrie diverse,Industrie diverse,Region Zentralschweiz,Region Zentralschweiz,Fahrzeuge/Handwerk/Lager/Transport,Lager,32,2017,six sigma tools ag junges schnell wachsend han...
7,7,26,jobsse-7512239,2017-04-28,2017-05-04,Assistentin Sales (m/w) - 100%,<p>Zu unseren Kunden zählen neben namhaften Pr...,Banken/ Finanzinstitute,Personalberatung,Region Zürich / Schaffhausen,Stadt Zürich / Zürichsee,Banking/Versicherungswesen,Anlageberatung/Private Banking,6,2017,kunde zählen neben namhaft privatbank angesehe...
8,8,32,jobsse-6140032,2014-11-25,2014-12-05,Stellvertretender Rayonverantwortlicher im Fri...,"<p>&nbsp;</p><p><span style=""font-size: 11pt;""...",Dienstleistungen allgemein,Beratung diverse,Region Zürich / Schaffhausen,Stadt Zürich / Zürichsee,Verkauf/Kundendienst/Innendienst,Verkauf Detailhandel,10,2014,aufgabenbereich gehörenvertretung abteilungsle...
9,9,33,jobsse-5872946,2014-06-16,2014-06-17,Lagerist und Magaziner im Bereich Schadstoffe ...,<p><strong>Ihr Aufgabengebiet</strong> umfasst...,Detail-/Grosshandel,Personalberatung,"Region Mittelland (AG, SO)","Region Mittelland (AG, SO)",Fahrzeuge/Handwerk/Lager/Transport,Lager,1,2014,aufgabengebiet umfassen kommissionieren rüsten...


In [32]:
data = data[data.lvl1_jobcategory_name == user_input]
if user_input_2 != "None":
    data = data[data.jobcategory_name == user_input_2]
data = data.drop('Unnamed: 0.1',axis=1)
data = data.drop('Unnamed: 0',axis=1)
print(data)

                  ad_id active_start_on active_end_on  \
2        jobsse-6058207      2014-10-06    2014-11-07   
25       jobsse-6316098      2015-03-18    2015-04-17   
26       jobsse-9837358      2020-08-18    2020-09-19   
34       jobsse-6048678      2014-09-30    2014-12-30   
56       jobsse-6863880      2016-03-29    2016-04-30   
...                 ...             ...           ...   
132727   jobsse-6512930      2015-07-28    2015-08-29   
132735   jobsse-7133366      2016-09-07    2016-09-14   
132747  jobsse-10651190      2021-09-09    2021-09-16   
132753   jobsse-7758470      2017-09-15    2017-10-12   
132764   jobsse-7011315      2016-06-15    2016-07-04   

                                                    title  \
2                Kaufmännische(n) Angestellte(n) 80%-100%   
25                          Leiter/in Qualitätsmanagement   
26             Partner als Co-Head Litigation/Arbitration   
34                                  Geschäftsführer (w/m)   
56        

In [33]:
# Abspeichern in ein CSV als Basis für das Wörterbuch
data.to_csv("Gesamtanalyse_gefiltert.csv", encoding="utf-8", sep=";")

In [9]:
# Vergleichen unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()
count = 0
while year <= data['year'].max():
    wordsets_phrases = data.loc[data['year'] == year]['Text_Anforderungen'] # Text Detok, da n-grams nach dem gesucht wurden
    for word in set_phrases['word']:
        count = sum(1 for s in wordsets_phrases if word in s)
        index = set_phrases[set_phrases['word']==word].index
        set_phrases.loc[index, year] = count
        set_phrases.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))    
    
    year += 1

In [10]:
set_fix = pd.read_excel("Fixwoerter.xlsx")

In [11]:
# Anforderungen Tokenisieren
set_fix['text_tok'] = set_fix.apply(lambda row: nltk.word_tokenize(row['word']), axis=1)
set_fix['text_tok'].head(10)

0      [abap]
1         [c]
2      [java]
3     [plsql]
4    [python]
5         [r]
6    [apache]
7     [azure]
8    [citrix]
9      [hana]
Name: text_tok, dtype: object

In [12]:
# Anforderungen in einem Set speichern
new_set = set()
set_fix['text_tok'].apply(new_set.update)
u = new_set

In [13]:
# Vergleiche unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()

while year <= data['year'].max():    
    wordsets_fix = [ frozenset(document.split(' ')) for document in data.loc[data['year'] == year]['Text_Anforderungen'] ]
    for word in u:
        count = sum( 1 for s in wordsets_fix if word in s )
        index = set_fix[set_fix['word']==word].index
        set_fix.loc[index, year] = count
        set_fix.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))

    year += 1

In [14]:
anforderungen = set_phrases.append(set_fix, ignore_index=True)

C:\Users\Sven Konermann\AppData\Local\Temp\ipykernel_74808\2611590460.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anforderungen = set_phrases.append(set_fix, ignore_index=True)


In [15]:
# Anforderungen mit den meisten Nennungen
anforderungen.sort_values(by=2021,ascending=False).head(5)

KeyError: 2021

In [ ]:
# Nur für Daten mit Jobcloud
anforderungen.to_csv("ausgewertete_Anforderungen.csv", sep=";")